In [4]:
import os
import numpy as np
import pandas as pd

from airpollution_trf_graph_loader import AirpollutionDatasetLoader

In [6]:
_T= 12 #target time horizon to analyze
_city= 'madrid'

loader= AirpollutionDatasetLoader(_city)
dataset=loader.get_dataset(T=_T)

feature_dim= loader.get_feature_dim()
feature_dim

{'trf': 4, 'ap0': 2, 'ap1': 5, 'ap2': 2, 'ap3': 1}

In [10]:
target_nodes= list(feature_dim.keys())
target_nodes

['trf', 'ap0', 'ap1', 'ap2', 'ap3']

In [12]:
y_hat_dict= {}
y_true_dict= {}

for k in target_nodes:
    _df= pd.read_csv(os.path.join('results',f'y_hat_{_city}_{_T}_{k}.csv'), index_col=0)
    y_hat_dict[k]= _df

    _df= pd.read_csv(os.path.join('results',f'y_true_{_city}_{_T}_{k}.csv'), index_col=0)
    y_true_dict[k]= _df

In [13]:
y_true_dict

{'trf':        Car  Motorcycle   Bus  Truck
 0     85.0         0.0   8.0    2.0
 1     14.0         0.0   0.0    0.0
 2    105.0         1.0   2.0    7.0
 3     38.0         4.0   0.0    0.0
 4     32.0        43.0   5.0    1.0
 ..     ...         ...   ...    ...
 177   71.0         2.0  15.0    9.0
 178    8.0         0.0   0.0    0.0
 179    3.0         0.0   0.0    1.0
 180   80.0         0.0   2.0   17.0
 181   48.0         1.0   2.0   30.0
 
 [182 rows x 4 columns],
 'ap0':      CO  SO2
 0   0.7  2.0
 1   0.6  1.0
 2   0.5  1.0
 3   0.4  1.0
 4   0.4  1.0
 5   0.4  1.0
 6   0.3  1.0
 7   0.3  1.0
 8   0.4  1.0
 9   0.4  1.0
 10  0.6  3.0
 11  0.8  3.0
 12  0.7  3.0
 13  0.5  2.0,
 'ap1':      CO    O3  PM10  PM25  SO2
 0   1.1   4.0  82.0  26.0  1.0
 1   1.3   4.0  65.0  22.0  1.0
 2   0.7   7.0  48.0  18.0  1.0
 3   0.2  45.0  34.0  17.0  1.0
 4   0.1  65.0  19.0  10.0  1.0
 5   0.1  52.0  24.0  12.0  1.0
 6   0.1  52.0  22.0  11.0  1.0
 7   0.1  52.0  16.0   7.0  1.0
 8   0.1 

In [ ]:
def mape_fn(actual, pred):
    #print(actual)
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+0.01))) * 100

def compute_metrics_as_dataframe_fn(y_valid, y_hat, particle_name):
    metrics= []
    metrics_global = {'mse':[],'rmse':[],'mae':[],'cvrmse':[],'mape':[],}

    try:
        mae = mean_absolute_error(y_valid, y_hat)
        mse = mean_squared_error(y_valid, y_hat)
        rmse= mean_squared_error(y_valid, y_hat, squared = False)
        cvrmse =  (rmse/np.mean(y_valid))*100 # it is a percentage
        mape = mape_fn(y_valid, y_hat)

        metrics.append((time_horizon, str(particle_name), mae, mse, rmse, cvrmse, mape))


        metrics_df = pd.DataFrame.from_records(metrics, columns='T particle MAE MSE RMSE CVRMSE MAPE'.split())

        return metrics_df
    except:
        return None

In [ ]:
for k in target_nodes:
    